# DataJoint U24 - Workflow Volume


## Interactively run the workflow


- If you haven't configured your set up, refer to [01-Configure](./01-Configure.ipynb).
- For an overview of the schema, refer to [02-WorkflowStructure](02-WorkflowStructure_Optional.ipynb).


Let's change the directory to load the local config, `dj_local_conf.json`.


In [ ]:
import os

# change to the upper level folder to detect dj_local_conf.json
if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir("..")

`pipeline.py` activates the various schema and declares other required tables.


In [ ]:
import datajoint as dj
from datetime import datetime
from element_zstack.export.bossdb import BossDBUpload
from workflow_zstack.pipeline import (
    lab,
    subject,
    session,
    scan,
    volume,
    bossdb,
    get_session_directory,
    get_volume_root_data_dir,
)

## Manually Inserting Entries


### Upstream tables


We can insert entries into `dj.Manual` tables (green in diagrams) by providing values as a dictionary or a list of dictionaries.


In [ ]:
subject.Subject.insert1(
    dict(subject="sub1", sex="M", subject_birth_date=datetime.now()),
    skip_duplicates=True,
)
session_key = dict(
    subject="sub1",
    session_id=1,
)
session.Session.insert1(
    dict(
        session_key,
        session_datetime=datetime.now(),
    ),
    skip_duplicates=True,
)
session.SessionDirectory.insert1(
    dict(**session.Session.fetch1("KEY"), session_dir=""),
    skip_duplicates=True,
)
scan.Scan.insert1(
    dict(
        session_key,
        scan_id=1,
        acq_software="ScanImage",
    ), skip_duplicates=True
)
scan_key = (scan.Scan & "subject = 'sub1'").fetch1("KEY")

In [ ]:
volume.Volume.populate(display_progress=True)

Next, we'll define a pameter set for segmentation with cellpose in the
`SegmentationParamset` table, and insert an entry into the `SegmentationTask`
table.  

In [ ]:
volume.SegmentationParamset.insert_new_params(
    segmentation_method="cellpose",
    paramset_idx=1,
    params=dict(
        diameter = 8,
        min_size = 2,
        do_3d = False,
        anisotropy = 0.5,
        model_type = "nuclei",
        channels = [[0, 0]],
        z_axis = 0,
        skip_duplicates=True,
    ),
)

In [ ]:
volume.SegmentationTask.insert1(
    dict(
        scan_key,
        paramset_idx=1,
        task_mode="trigger",
        ), skip_duplicates=True
)

Now, we can popluate the `Segmentation` table.

In [ ]:
volume.Segmentation.populate()

Finally, we can upload our data either from the data stored in the table or a path to images. If this entry is already associated with a `SessionDirectory` entry, we'll look for images in this path.


In [ ]:
bossdb.UploadParamSet.insert_new_params(
    paramset_idx=1,
    paramset_desc="test params",
    params=dict(
        voxel_units="micrometers",
        voxel_size=[1,1,1],
    )
)

In [ ]:
bossdb.VolumeUploadTask()

In [ ]:
col_name = "dataJointTestUpload"
exp_name = "CaImaging"
chn_name = "test10"
bossdb.VolumeUploadTask.update1(
    dict(
        scan_key,
        paramset_idx=1,
        collection_name=col_name,
        experiment_name=exp_name,
        channel_name=chn_name,
        upload_type="image",
    )
)

In [ ]:
bossdb.VolumeUploadTask()

In [ ]:
upload_key = (bossdb.VolumeUploadTask & scan_key & "channel_name = 'test10'").fetch("KEY")

In [ ]:
upload_key

In [ ]:
bossdb.BossDBURLs.populate(upload_key)

In [ ]:
(bossdb.BossDBURLs & scan_key).fetch1("neuroglancer_url")

Explain how to get permission from APL to upload data. 

Create a schema to automatically generate neuroglancer link and insert into DJ
table. 

Include BossDBUpload in BossDBURLs as a computed/imported table. 